# R.O.P Example: "ack" command wrapper using shellCommandROP

This is an example of how to "wrap" a unix command with shellCommandROP.

## TL;DR
Wrapper of "ack" command made with shellCommand.

## Prerequisites

You need "ack" installed in your machine:

> sudo apt-get install ack

## ROP and shellCommandROP: everything is there

In [1]:
%install-location $cwd/swift-install
%install '.package(path: "$cwd/..")' Rop

Installing packages:
	.package(path: "/notebooks/swift-rop/jupyter/..")
		Rop
With SwiftPM flags: []
Working in: /tmp/tmpkbaaov68/swift-install
Compile Swift Module 'jupyterInstalledPackages' (1 sources)
Initializing Swift...
Installation complete!


In [2]:
import Rop

# Ack wrapper

### Support functions

In [3]:
//export 
func unixPwd() -> String {
    return shell("/bin/pwd")    
        .then{pwd in pwd.replacingOccurrences(of: "\n", with: "")} // Clean pwd output form extra "\n"
        .value! // Should be safe: usually pwd is not null ;-)
}

In [4]:
print("'\(unixPwd())'")

'/notebooks/swift-rop/jupyter'


In [5]:
//export 

//act as doc in functional languages ;-) 
typealias AckResult = (fileName: String, row: Int, sample: String)

func processLine(_ src: String) -> AckResult {
    let tmp = src.split(separator: ":", maxSplits: 2, omittingEmptySubsequences: true)
    if tmp.count >= 3 {
        let file = String(tmp[0])
        let line = Int(tmp[1])! // Prtty sure it's a number
        let sample = String(tmp[2])
        return (fileName: file, row: line, sample: sample)
    } else {
        return (fileName: "?", row: 0, sample: "?")
    }
}

In [6]:
func test_processLine() {
    let sampleLine = "/notebooks/fastai_docs/dev_swift/FastaiNotebook_08_data_block/Sources/FastaiNotebook_08_data_block/04_callbacks.swift:72:/// A model learner, responsible for initializing and training a model on a given dataset."
    print("IN:", sampleLine)
    print("OUT:", processLine(sampleLine))
    print("OUT WITH ERRORS:", processLine("asdas dasd adas dasdas"))
}
test_processLine()

IN: /notebooks/fastai_docs/dev_swift/FastaiNotebook_08_data_block/Sources/FastaiNotebook_08_data_block/04_callbacks.swift:72:/// A model learner, responsible for initializing and training a model on a given dataset.
OUT: (fileName: "/notebooks/fastai_docs/dev_swift/FastaiNotebook_08_data_block/Sources/FastaiNotebook_08_data_block/04_callbacks.swift", row: 72, sample: "/// A model learner, responsible for initializing and training a model on a given dataset.")
OUT WITH ERRORS: (fileName: "?", row: 0, sample: "?")


### ack implementation

In [7]:
//export
import Foundation

typealias AckResults = Result<[AckResult]>

func printAckResults(ackrRes: AckResults, printFullPath: Bool) {
    // Should check for errors...
    if let ackr=ackrRes.value {
        for row in ackr {
            let theFileName = (printFullPath) ? row.fileName : (row.fileName as NSString).lastPathComponent
            print(theFileName, "R:\(row.row)", row.sample)
        }
    } else {
        print(">> No results found <<")
    }
}

func ack(word: String, path: String? = nil, caseInsensitive: Bool = true, show: Bool = true, printFullPath: Bool = false) -> AckResults {
    let thePath = path ?? unixPwd()
    let args = good([word, thePath])
        .then{args in caseInsensitive ? args + ["-i"] : args}
    let ret = shell("/usr/bin/ack", args.value!)
        .then{str in str.components(separatedBy: "\n")}
        //.use{lines in print(lines)} // Debug
        .then{lines in lines.map(processLine)}    
        .then{rets in rets.filter{r in r.fileName != "?"}} // Filtering out errors
        .then{rets in (rets.count>0) ? good(rets) : bad("No results found")} // No resuls is "bad" ;-) 
    if show {
        if ret.isOk() {
            printAckResults(ackrRes: ret, printFullPath: printFullPath)
        }
    }
    return ret
}

Let's make some tests...

In [8]:
ack(word: "float") // Search in current folder

: 

In [9]:
ack(word: "model", path: "/notebooks/fastai_docs/dev_swift/FastaiNotebook_08_data_block") // Search in folder

08_data_block.swift R:251 public struct CNNModel: Layer {
06_cuda.swift R:21 public struct CnnModel: Layer {
07_batchnorm.swift R:118 public struct CnnModelBN: Layer {
04_callbacks.swift R:10 public struct BasicModel: Layer {
04_callbacks.swift R:72 /// A model learner, responsible for initializing and training a model on a given dataset.
04_callbacks.swift R:76           Opt.Model: Layer,
04_callbacks.swift R:77           // Constrain model input to Tensor<Float>, to work around
04_callbacks.swift R:79           Opt.Model.Input == Tensor<Float>
04_callbacks.swift R:82     public typealias Model = Opt.Model
04_callbacks.swift R:83     public typealias Input = Model.Input
04_callbacks.swift R:84     public typealias Output = Model.Output
04_callbacks.swift R:88     public typealias Variables = Model.AllDifferentiableVariables
04_callbacks.swift R:94         public typealias F = @differentiable (Model.Output, @nondiff Label) -> Loss
04_callbacks.swift R:99     /// The dataset on which th

▿ Result<Array<(fileName: String, row: Int, sample: String)>>
  ▿ Ok : 31 elements
    ▿ 0 : 3 elements
      - fileName : "/notebooks/fastai_docs/dev_swift/FastaiNotebook_08_data_block/Sources/FastaiNotebook_08_data_block/08_data_block.swift"
      - row : 251
      - sample : "public struct CNNModel: Layer {"
    ▿ 1 : 3 elements
      - fileName : "/notebooks/fastai_docs/dev_swift/FastaiNotebook_08_data_block/Sources/FastaiNotebook_08_data_block/06_cuda.swift"
      - row : 21
      - sample : "public struct CnnModel: Layer {"
    ▿ 2 : 3 elements
      - fileName : "/notebooks/fastai_docs/dev_swift/FastaiNotebook_08_data_block/Sources/FastaiNotebook_08_data_block/07_batchnorm.swift"
      - row : 118
      - sample : "public struct CnnModelBN: Layer {"
    ▿ 3 : 3 elements
      - fileName : "/notebooks/fastai_docs/dev_swift/FastaiNotebook_08_data_block/Sources/FastaiNotebook_08_data_block/04_callbacks.swift"
      - row : 10
      - sample : "public struct BasicModel: Layer {"
   

In [10]:
// Search case sensitive
ack(word: "float", path: "/notebooks/fastai_docs/dev_swift/FastaiNotebook_08_data_block", caseInsensitive: false)

05b_early_stopping.swift R:25         // A learning rate schedule from step to float.
04_callbacks.swift R:123     /// The current epoch + iteration, float between 0.0 and epochCount
05_anneal.swift R:150         // A learning rate schedule from step to float.


▿ Result<Array<(fileName: String, row: Int, sample: String)>>
  ▿ Ok : 3 elements
    ▿ 0 : 3 elements
      - fileName : "/notebooks/fastai_docs/dev_swift/FastaiNotebook_08_data_block/Sources/FastaiNotebook_08_data_block/05b_early_stopping.swift"
      - row : 25
      - sample : "        // A learning rate schedule from step to float."
    ▿ 1 : 3 elements
      - fileName : "/notebooks/fastai_docs/dev_swift/FastaiNotebook_08_data_block/Sources/FastaiNotebook_08_data_block/04_callbacks.swift"
      - row : 123
      - sample : "    /// The current epoch + iteration, float between 0.0 and epochCount"
    ▿ 2 : 3 elements
      - fileName : "/notebooks/fastai_docs/dev_swift/FastaiNotebook_08_data_block/Sources/FastaiNotebook_08_data_block/05_anneal.swift"
      - row : 150
      - sample : "        // A learning rate schedule from step to float."


In [11]:
// Full path on result
ack(word: "float", path: "/notebooks/fastai_docs/dev_swift/FastaiNotebook_08_data_block", caseInsensitive: false, printFullPath: true)

/notebooks/fastai_docs/dev_swift/FastaiNotebook_08_data_block/Sources/FastaiNotebook_08_data_block/05b_early_stopping.swift R:25         // A learning rate schedule from step to float.
/notebooks/fastai_docs/dev_swift/FastaiNotebook_08_data_block/Sources/FastaiNotebook_08_data_block/04_callbacks.swift R:123     /// The current epoch + iteration, float between 0.0 and epochCount
/notebooks/fastai_docs/dev_swift/FastaiNotebook_08_data_block/Sources/FastaiNotebook_08_data_block/05_anneal.swift R:150         // A learning rate schedule from step to float.


▿ Result<Array<(fileName: String, row: Int, sample: String)>>
  ▿ Ok : 3 elements
    ▿ 0 : 3 elements
      - fileName : "/notebooks/fastai_docs/dev_swift/FastaiNotebook_08_data_block/Sources/FastaiNotebook_08_data_block/05b_early_stopping.swift"
      - row : 25
      - sample : "        // A learning rate schedule from step to float."
    ▿ 1 : 3 elements
      - fileName : "/notebooks/fastai_docs/dev_swift/FastaiNotebook_08_data_block/Sources/FastaiNotebook_08_data_block/04_callbacks.swift"
      - row : 123
      - sample : "    /// The current epoch + iteration, float between 0.0 and epochCount"
    ▿ 2 : 3 elements
      - fileName : "/notebooks/fastai_docs/dev_swift/FastaiNotebook_08_data_block/Sources/FastaiNotebook_08_data_block/05_anneal.swift"
      - row : 150
      - sample : "        // A learning rate schedule from step to float."


In [12]:
ack(word: "xxxx" + String(Int.random(in: 0 ..< 10))) // No result

▿ Result<Array<(fileName: String, row: Int, sample: String)>>
  ▿ Error : ShellCommandError
    - code : 1
    - msg : ""


In [13]:
ack(word: "xxxx", path: "/asd/asd") // Wrong folder

▿ Result<Array<(fileName: String, row: Int, sample: String)>>
  ▿ Error : ShellCommandError
    - code : 1
    - msg : "ack: /asd/asd: No such file or directory\n"


In [14]:
//No output
let ret = ack(word: "model", path: "/notebooks/fastai_docs/dev_swift/FastaiNotebook_08_data_block", show: false)
print("Was last result successfull??", ret.isOk())

Was last result successfull?? true
